Here, we present a variety of functions for working with the toric ideals of neural codes. In this context, the toric ideal of a neural code $\mathcal C$ is the toric ideal of a matrix whose columns are the (nonzero) codewords of $\mathcal C$. First, note that Sage can find the toric ideal of a matrix easily:

In [1]:
mat = matrix([[0,0,0,1], [0,0,1,1], [0,0,1,0], [0,1,1,0], [0,1,0,0], [1,1,0,0],[1,0,0,0]]).transpose()
ToricIdeal(mat)


Ideal (-z4*z6 + z5, -z2*z4 + z3, z0*z2 - z1) of Multivariate Polynomial Ring in z0, z1, z2, z3, z4, z5, z6 over Rational Field

The default variable names, $z_1, z_2,\ldots,z_m $ are not particularly informative, and thanks to a strange bug in the way Sage calls either 4ti2 or GFan, actually cause some errors. Thus, we want to relabel the variables so that they are labeled with the codeword they represent for substantial as well as aesthetic reasons. Here's a function that does that: 

In [2]:
#input: a matrix in which columns are codewords
#output: a string specifying variable names yc, where c is the binary number corresponding to a codeword
#example:  make_variable_names_from_matrix([1,0]//[0,1]) should return 'y10, y01'
def make_variable_names_from_matrix(mat):
    names = ''
    for col in mat.columns():
        name = 'y'
        for position in col:
            name=name+(str(position))
        names=names+name+ ',' 
    return names[:-1]


In [3]:
make_variable_names_from_matrix(mat)

'y0001,y0011,y0010,y0110,y0100,y1100,y1000'

In [4]:
ToricIdeal(mat, make_variable_names_from_matrix(mat))

Ideal (-y0100*y1000 + y1100, -y0010*y0100 + y0110, y0001*y0010 - y0011) of Multivariate Polynomial Ring in y0001, y0011, y0010, y0110, y0100, y1100, y1000 over Rational Field

If we want to investigate toric ideals of inductively pierced neural codes, it is nice to have a way to make a lot of them. Here's code that does this--given $n$, it spits out all inductively 1 pierced codes on $n$ neurons.

In [5]:
#return the list of all inductively pierced codes on n neurons (may have some redundancy)
def make_induct_one_pierced(n):
    if n==1:
        return [[[0],[1]]]
    else: 
        #recursive call to produce a list of all inductively pierced codes on n-1 neurons
        smaller_codes=make_induct_one_pierced(n-1)
        zero_piercings=[]
        one_piercings=[]
        #for each code on n-1 neurons, construct each 0 and 1-piercing of the code
        for code in smaller_codes:
            #make a version of the code with a zero added to the end of every codeword
            inflate=[]
            for codeword in code:
                temp=list(codeword)
                temp.append(0)
                inflate.append(temp)
            #make all 0-piercings--to do this, we should do a zero piercing wrt to each codeword as a background zone
            for d in code:
                new=list(d)
                new.append(1)
                newcode=list(inflate)
                newcode.append(new)
                zero_piercings.append(newcode)
            # now, go through all of the n-1 neurons in the code    
            for i in range (0, n-1):
                chi=[]
                pairs=[]
                for codeword in code:
                    if codeword[i]==1:
                        chi.append(codeword)
                for codeword in chi:
                    partner1=list(codeword)
                    partner2=list(codeword)
                    partner2[i]=0
                    if partner2 in code:
                        pairs.append((partner1, partner2))
                #find each possible way to pierce that neuron       
                for pair in pairs:
                    pair[0].append(1)
                    pair[1].append(1)
                    newcode=list(inflate)
                    newcode.extend([ pair[1], pair[0]])
                    one_piercings.append(newcode)
        return zero_piercings+one_piercings

For instance, here are all inductively pierced neural codes on 3 neurons. 

In [6]:
make_induct_one_pierced(3)

[[[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]],
 [[0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0, 1]],
 [[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 1]],
 [[0, 0, 0], [1, 0, 0], [1, 1, 0], [0, 0, 1]],
 [[0, 0, 0], [1, 0, 0], [1, 1, 0], [1, 0, 1]],
 [[0, 0, 0], [1, 0, 0], [1, 1, 0], [1, 1, 1]],
 [[0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0], [0, 0, 1]],
 [[0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0], [1, 0, 1]],
 [[0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0], [0, 1, 1]],
 [[0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0], [1, 1, 1]],
 [[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 1]],
 [[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [0, 1, 1]],
 [[0, 0, 0], [1, 0, 0], [1, 1, 0], [0, 0, 1], [1, 0, 1]],
 [[0, 0, 0], [1, 0, 0], [1, 1, 0], [1, 0, 1], [1, 1, 1]],
 [[0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0], [0, 0, 1], [1, 0, 1]],
 [[0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0], [0, 1, 1], [1, 1, 1]],
 [[0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0], [0, 0, 1], [0, 1, 1]],
 [[0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0], [

Now, we want to comptue the toric ideals of all of our one pierced codes, their Groebner bases in our term order, and the maximum degree of their Groebner basis in any term order. We do this with the function make_table_toric_degrees(code_list). In order for the default lexicographic order used by sage to agree with our term order, we first reverse the codes.  

In [10]:
def reverse_code(code):
    results = [code[0]]
    for i in range (0, len(code)-1):
        results.append(code[len(code)-1-i])
    return results 
def make_table_toric_degrees(code_list):
    matrices = []
    toric_ideals = []
    nice_formatting =[]
    clean = []
    for code in code_list:
        mat=matrix(reverse_code(code)[1:]).transpose()
        toric_ideal=ToricIdeal(mat, make_variable_names_from_matrix(mat)) 
        toric_ideals.append(toric_ideal)
        degrees = list(map(lambda x: x.degree(), toric_ideal.gens()))
        if degrees !=[-1]:
            clean.append([mat, toric_ideal.groebner_basis(), max(degrees)])
    return table(clean)
        

In [11]:
make_table_toric_degrees(make_induct_one_pierced(3))

,,
,,
,,
,,
,,
,,
,,
,,
,,
,,
,,
